# import Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
from pyproj import Transformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv("Divar.csv")

In [ ]:
data.columns

In [ ]:
data['transformable_price'].value_counts

In [ ]:

data.loc[
    data['construction_year'].astype(str).str.contains('قبل از', na=False),
    'construction_year'
] = 1369

#created_at_month to dateTime
data['created_at_month']=pd.to_datetime(data['created_at_month'],errors='coerce')
data['year_month'] = data['created_at_month'].dt.to_period('M')

In [ ]:
def persian_to_english_numbers(text):
    if not isinstance(text, str):
        return text

    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    arabic_digits  = '٠١٢٣٤٥٦٧٨٩'
    english_digits = '0123456789'

    translation_table = str.maketrans(
        persian_digits + arabic_digits,
        english_digits * 2
    )

    return text.translate(translation_table)

In [ ]:
data=data.applymap(persian_to_english_numbers)

In [ ]:
data['created_at_month']=pd.to_datetime(data['created_at_month'],errors='coerce')
data['year_month'] = data['created_at_month'].dt.to_period('M')

In [ ]:
# ---------- 3. تبدیل rooms_count به عدد ----------
def parse_rooms(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().lower()
    if 'بدون' in s or 'صفر' in s:
        return 0
    if 'یک' in s:
        return 1
    if 'دو' in s:
        return 2
    if 'سه' in s:
        return 3
    if 'چهار' in s:
        return 4
    if 'پنج' in s or 'بیشتر' in s:
        return 5
    nums = re.findall(r'\d+', s)
    return int(nums[0]) if nums else np.nan

# Part 1 old

In [ ]:
data.columns

In [ ]:
data[['price_value','transformed_rent']].head()

In [ ]:
# --------------------------------------------
# 🔧 خوشه‌بندی املاک — نسخه نهایی (بدون outlier)
# --------------------------------------------

df = data.copy()


df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. محاسبه transformable_price ----------
df['rent_val'] = pd.to_numeric(df['rent_value'], errors='coerce')
df['cred_val'] = pd.to_numeric(df['credit_value'], errors='coerce')
df['price_val'] = pd.to_numeric(df['price_value'], errors='coerce')

def calc_transformable(row):
    if 'sell' in str(row['cat2_slug']).lower() and row['price_val'] > 0:
        return row['price_val']
    if row['cred_val'] > 0 or row['rent_val'] > 0:
        return (row['cred_val'] or 0) + (row['rent_val'] or 0) * 1200
    return np.nan

df['tp'] = df.apply(calc_transformable, axis=1)

# ---------- 5. فیلتر اولیه: فقط رکوردهای معتبر ----------
df = df[
    (df['tp'].notna()) & (df['tp'] > 0) &
    (df['building_size'].notna()) & (df['building_size'] > 5) &
    (df['location_latitude'].notna()) & (df['location_longitude'].notna())
].copy()

print(f"✅ پس از فیلتر اولیه: {len(df)} رکورد")

# ---------- 6. تبدیل مختصات به UTM (Zone 39N) ----------
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32639")
valid_mask = (
    df['location_latitude'].between(25, 40) &
    df['location_longitude'].between(44, 63)
)
df = df[valid_mask]

lat_utm, lon_utm = transformer.transform(
    df['location_latitude'].values,
    df['location_longitude'].values
)
df['lat_utm'] = lat_utm
df['lon_utm'] = lon_utm

# ---------- 7. حذف outlierهای شدید (حیاتی!) ----------
print(f"قبل از فیلتر outlier: max size = {df['building_size'].max():.0f}, max price = {df['tp'].max():.0f}")

# 🚫 حذف:
# - زیربنا > 50,000 متر (غیرواقعی برای مسکونی)
# - قیمت > 200 میلیارد تومان (پرت شدید)
# - زیربنا < 10 متر (خراب)
df_clean = df[
    (df['building_size'] > 10) &
    (df['building_size'] <= 50_000) &
    (df['tp'] <= 200_000_000_000)  # 200 میلیارد
]

print(f"✅ پس از حذف outlier: {len(df_clean)} رکورد")
print(f"   → max size = {df_clean['building_size'].max():.0f}, max price = {df_clean['tp'].max():.0f}")

# ---------- 8. ساخت ویژگی‌های نهایی ----------
# امتیاز امکانات (0–3)
for col in ['has_parking', 'has_elevator', 'has_security_guard']:
    df_clean[col] = df_clean[col].astype(str).str.lower().isin(['true', '1', 'yes'])
df_clean['amenities'] = (
    df_clean['has_parking'].astype(int) +
    df_clean['has_elevator'].astype(int) +
    df_clean['has_security_guard'].astype(int)
)

features = [
    'tp',               # قیمت تبدیل‌شده
    'building_size',   # زیربنا
    'rooms_num',       # تعداد اتاق
    'lat_utm',         # مختصات
    'lon_utm',
    'amenities'        # امتیاز امکانات
]

X = df_clean[features].copy()

# ---------- 9. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 10. PCA (90% واریانس) ----------
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_comp = np.argmax(cumsum >= 0.90) + 1

pca = PCA(n_components=n_comp, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"\n📊 PCA: {n_comp} مؤلفه برای 90% واریانس (واریانس: {cumsum[n_comp-1]:.1%})")
print("وزن‌های مؤلفه اول (PC1):")
pc1_weights = pd.Series(pca.components_[0], index=features).round(3)
print(pc1_weights)

# ---------- 11. KMeans (K=10 — منطقی برای 2000 رکورد) ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=20)
clusters = kmeans.fit_predict(X_pca)
df_clean['cluster'] = clusters

# ---------- 12. رسم نمودار ----------
df_plot = df_clean.copy()
df_plot['PC1'] = X_pca[:, 0]
df_plot['PC2'] = X_pca[:, 1] if X_pca.shape[1] > 1 else 0
df_plot['price_mil'] = df_plot['tp'] / 1_000_000

fig = px.scatter(
    df_plot,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={'PC1': 'مؤلفه اصلی ۱', 'PC2': 'مؤلفه اصلی ۲'},
    title=f'خوشه‌بندی KMeans (K=10) روی فضای PCA — {len(df_plot)} رکورد پس از حذف outlier',
    opacity=0.7,
    width=950, height=600
)

# مراکز خوشه‌ها
centers_pca = kmeans.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers_pca[:, 0], 
    y=centers_pca[:, 1] if centers_pca.shape[1] > 1 else [0]*len(centers_pca),
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(5)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=16)
fig.write_html("kmeans_final.html")
fig.show()

# ---------- 13. خلاصه خوشه‌ها ----------
summary = df_plot.groupby('cluster').agg(
    count=('tp', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('building_size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_amenities=('amenities', 'mean'),
    main_city=('city_slug', lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else '—')
).round(2)

print("\n📊 خلاصه نهایی خوشه‌ها:")
print(summary.sort_values('avg_price_mil', ascending=False))

In [ ]:
# --------------------------------------------------
# ✅ KMeans با K = 10 — مطابق صورت سوال
#    بدون PCA، با حذف outlierها و scaling صحیح
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from pyproj import Transformer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go


df = data.copy()





df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. محاسبه transformable_price ----------
# df['rent_val'] = pd.to_numeric(df['rent_value'], errors='coerce')
# df['cred_val'] = pd.to_numeric(df['credit_value'], errors='coerce')
df['price_val'] = pd.to_numeric(df['price_value'], errors='coerce')

# def calc_tp(row):
#     # فروش
#     if 'sell' in str(row['cat2_slug']).lower() and row['price_val'] > 0:
#         return row['price_val']
#     # رهن + اجاره
#     if row['cred_val'] > 0 or row['rent_val'] > 0:
#         return (row['cred_val'] or 0) + (row['rent_val'] or 0) * 1200
#     # فقط اجاره
#     if row['rent_val'] > 0:
#         return row['rent_val'] * 1200
#     return np.nan

# df['tp'] = df.apply(calc_tp, axis=1)

# ---------- 5. فیلتر اولیه ----------
df = df[
    (df['price_val'].notna()) & (df['price_val'] > 10_000_000) &  # ≥10 میلیون تومان
    (df['building_size'].notna()) & (df['building_size'].between(10, 50_000)) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
].copy()

print(f"✅ پس از فیلتر اولیه: {len(df)} رکورد")

# ---------- 6. حذف outlierهای شدید (99.5th percentile) ----------
upper_price = df['price_val'].quantile(0.995)   # ~50 میلیارد
upper_size  = df['building_size'].quantile(0.995)  # ~2000 متر

df_clean = df[
    (df['price_val'] <= upper_price) &
    (df['building_size'] <= upper_size)
]

print(f"✅ پس از حذف outlier: {len(df_clean)} رکورد")

# ---------- 7. تبدیل مختصات به UTM (Zone 39N) ----------
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32639")
lat_utm, lon_utm = transformer.transform(
    df_clean['location_latitude'].values,
    df_clean['location_longitude'].values
)
df_clean['lat_utm'] = lat_utm
df_clean['lon_utm'] = lon_utm

# ---------- 8. ساخت امتیاز امکانات ----------
for col in ['has_parking', 'has_elevator', 'has_security_guard']:
    df_clean[col] = df_clean[col].astype(str).str.lower().isin(['true', '1', 'yes'])
df_clean['amenities'] = df_clean[['has_parking', 'has_elevator', 'has_security_guard']].sum(axis=1)

# ---------- 9. انتخاب ویژگی‌ها ----------
features = [
    'price_val',               # قیمت تبدیل‌شده (تومان)
    'building_size',   # زیربنا (متر)
    'rooms_num',       # تعداد اتاق
    'lat_utm',         # UTM
    'lon_utm',
    'amenities'        # 0–3
]

X = df_clean[features].copy()

# ---------- 10. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 11. ✅ KMeans با دقیقاً K = 10 ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_scaled)
df_clean['cluster'] = clusters

# ---------- 12. رسم: قیمت vs زیربنا (با 10 خوشه) ----------
df_clean['price_mil'] = df_clean['price_val'] / 1_000_000

fig = px.scatter(
    df_clean,
    x='building_size',
    y='price_mil',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={
        'building_size': 'زیربنا (متر مربع)',
        'price_mil': 'قیمت تبدیل‌شده (میلیون تومان)',
        'cluster': 'خوشه'
    },
    title='خوشه‌بندی KMeans — دقیقاً ۱۰ خوشه (بدون PCA)',
    opacity=0.65,
    width=950, height=650
)

# مراکز خوشه‌ها (در فضای اصلی)
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)
centers_df = pd.DataFrame(centers_original, columns=features)

fig.add_trace(go.Scatter(
    x=centers_df['building_size'],
    y=centers_df['price_val'] / 1_000_000,
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(10)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=18)
fig.write_html("kmeans_k10_no_pca.html")
fig.show()

# ---------- 13. جدول خلاصه (10 خوشه) ----------
summary = df_clean.groupby('cluster').agg(
    count=('price_val', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('building_size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_amenities=('amenities', 'mean'),
    main_city=('city_slug', lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else '—')
).round(2)

print("\n📊 خلاصه ۱۰ خوشه (مرتب‌شده بر اساس قیمت):")
print(summary.sort_values('avg_price_mil', ascending=False))

# ---------- 14. نکته تحلیلی ----------
print("\n💡 نکته:")
print("- برخی خوشه‌ها ممکن است کوچک باشند (مثل خوشه‌های ویژه: 'ویلاهای ساحلی با استخر')")
print("- این طبیعی است — چون K=10 از قبل تعیین شده و ممکن است برخی خوشه‌ها تخصصی باشند.")

In [ ]:
# --------------------------------------------
# 🔧 خوشه‌بندی املاک — نسخه نهایی (بدون outlier)
# --------------------------------------------

df = data.copy()


df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. محاسبه transformable_price ----------
# df['rent_val'] = pd.to_numeric(df['rent_value'], errors='coerce')
# df['cred_val'] = pd.to_numeric(df['credit_value'], errors='coerce')
df['price_val'] = pd.to_numeric(df['price_value'], errors='coerce')

# def calc_transformable(row):
#     if 'sell' in str(row['cat2_slug']).lower() and row['price_val'] > 0:
#         return row['price_val']
#     if row['cred_val'] > 0 or row['rent_val'] > 0:
#         return (row['cred_val'] or 0) + (row['rent_val'] or 0) * 1200
#     return np.nan

# df['tp'] = df.apply(calc_transformable, axis=1)
df['tp']=df['price_val'] 
# ---------- 5. فیلتر اولیه: فقط رکوردهای معتبر ----------
df = df[
    (df['tp'].notna()) & (df['tp'] > 0) &
    (df['building_size'].notna()) & (df['building_size'] > 5) &
    (df['location_latitude'].notna()) & (df['location_longitude'].notna())
].copy()

print(f"✅ پس از فیلتر اولیه: {len(df)} رکورد")

# ---------- 6. تبدیل مختصات به UTM (Zone 39N) ----------
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32639")
valid_mask = (
    df['location_latitude'].between(25, 40) &
    df['location_longitude'].between(44, 63)
)
df = df[valid_mask]

lat_utm, lon_utm = transformer.transform(
    df['location_latitude'].values,
    df['location_longitude'].values
)
df['lat_utm'] = lat_utm
df['lon_utm'] = lon_utm

# ---------- 7. حذف outlierهای شدید (حیاتی!) ----------
print(f"قبل از فیلتر outlier: max size = {df['building_size'].max():.0f}, max price = {df['tp'].max():.0f}")

# 🚫 حذف:
# - زیربنا > 50,000 متر (غیرواقعی برای مسکونی)
# - قیمت > 200 میلیارد تومان (پرت شدید)
# - زیربنا < 10 متر (خراب)
df_clean = df[
    (df['building_size'] > 10) &
    (df['building_size'] <= 50_000) &
    (df['tp'] <= 200_000_000_000)  # 200 میلیارد
]

print(f"✅ پس از حذف outlier: {len(df_clean)} رکورد")
print(f"   → max size = {df_clean['building_size'].max():.0f}, max price = {df_clean['tp'].max():.0f}")

# ---------- 8. ساخت ویژگی‌های نهایی ----------
# امتیاز امکانات (0–3)
for col in ['has_parking', 'has_elevator', 'has_security_guard']:
    df_clean[col] = df_clean[col].astype(str).str.lower().isin(['true', '1', 'yes'])
df_clean['amenities'] = (
    df_clean['has_parking'].astype(int) +
    df_clean['has_elevator'].astype(int) +
    df_clean['has_security_guard'].astype(int)
)

features = [
    'tp',               # قیمت تبدیل‌شده
    'building_size',   # زیربنا
    'rooms_num',       # تعداد اتاق
    'lat_utm',         # مختصات
    'lon_utm',
    'amenities'        # امتیاز امکانات
]

X = df_clean[features].copy()

# ---------- 9. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 10. PCA (90% واریانس) ----------
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_comp = np.argmax(cumsum >= 0.90) + 1

pca = PCA(n_components=n_comp, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"\n📊 PCA: {n_comp} مؤلفه برای 90% واریانس (واریانس: {cumsum[n_comp-1]:.1%})")
print("وزن‌های مؤلفه اول (PC1):")
pc1_weights = pd.Series(pca.components_[0], index=features).round(3)
print(pc1_weights)

# ---------- 11. KMeans (K=10 — منطقی برای 2000 رکورد) ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=20)
clusters = kmeans.fit_predict(X_pca)
df_clean['cluster'] = clusters

# ---------- 12. رسم نمودار ----------
df_plot = df_clean.copy()
df_plot['PC1'] = X_pca[:, 0]
df_plot['PC2'] = X_pca[:, 1] if X_pca.shape[1] > 1 else 0
df_plot['price_mil'] = df_plot['tp'] / 1_000_000

fig = px.scatter(
    df_plot,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={'PC1': 'مؤلفه اصلی ۱', 'PC2': 'مؤلفه اصلی ۲'},
    title=f'خوشه‌بندی KMeans (K=10) روی فضای PCA — {len(df_plot)} رکورد پس از حذف outlier',
    opacity=0.7,
    width=950, height=600
)

# مراکز خوشه‌ها
centers_pca = kmeans.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers_pca[:, 0], 
    y=centers_pca[:, 1] if centers_pca.shape[1] > 1 else [0]*len(centers_pca),
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(5)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=16)
fig.write_html("kmeans_final.html")
fig.show()

# ---------- 13. خلاصه خوشه‌ها ----------
summary = df_plot.groupby('cluster').agg(
    count=('tp', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('building_size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_amenities=('amenities', 'mean'),
    main_city=('city_slug', lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else '—')
).round(2)

print("\n📊 خلاصه نهایی خوشه‌ها:")
print(summary.sort_values('avg_price_mil', ascending=False))

In [ ]:
# --------------------------------------------------
# ✅ KMeans با K=10 — فقط آگهی‌های فروشی + ویژگی‌های هوشمندانه
#    ✅ فقط price_value (نه transformable_price)
#    ✅ سال ساخت و شهر به عنوان فیچرهای کلیدی
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from pyproj import Transformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go


df = data.copy()

# ---------- 2. تبدیل اعداد فارسی ----------
def persian_to_english_digits(text):
    if not isinstance(text, str): return text
    return text.translate(str.maketrans("۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩", "01234567890123456789"))


# ---------- 3. تبدیل rooms_count ----------
def parse_rooms(val):
    if pd.isna(val): return np.nan
    s = str(val).strip().lower()
    if 'بدون' in s: return 0
    if 'یک' in s: return 1
    if 'دو' in s: return 2
    if 'سه' in s: return 3
    if 'چهار' in s: return 4
    if 'پنج' in s: return 5
    nums = re.findall(r'\d+', s)
    return int(nums[0]) if nums else np.nan

df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. پاک‌سازی سال ساخت ----------
def parse_year(val):
    if pd.isna(val): return np.nan
    s = str(val).strip()
    s = persian_to_english_digits(s)
    nums = re.findall(r'1[3-4]\d{2}', s)  # سال 1300 تا 1499
    return int(nums[0]) if nums else np.nan

df['year'] = df['construction_year'].apply(parse_year)

# ---------- 5. فیلتر فقط آگهی‌های فروشی ----------
df_sell = df[
    (df['cat2_slug'].str.contains('sell', case=False, na=False)) &
    (df['price_value'].notna()) &
    (df['building_size'].notna()) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
].copy()

# ---------- 6. تبدیل عددی + فیلتر منطقی ----------
df_sell['price'] = pd.to_numeric(df_sell['price_value'], errors='coerce')
df_sell['size'] = pd.to_numeric(df_sell['building_size'], errors='coerce')

df_sell = df_sell[
    (df_sell['price'] > 10_000_000) & (df_sell['price'] <= 200_000_000_000) &
    (df_sell['size'] > 10) & (df_sell['size'] <= 50_000) &
    (df_sell['year'].between(1300, 1405))  # فقط سال‌های معقول
]

print(f"✅ آگهی‌های فروشی معتبر: {len(df_sell)}")

# ---------- 7. تبدیل مختصات به UTM ----------
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32639")
lat_utm, lon_utm = transformer.transform(
    df_sell['location_latitude'].values,
    df_sell['location_longitude'].values
)
df_sell['lat_utm'] = lat_utm
df_sell['lon_utm'] = lon_utm

# ---------- 8. کدگذاری شهر (LabelEncoder) ----------
le_city = LabelEncoder()
# فقط شهرهای با ≥10 آگهی
city_counts = df_sell['city_slug'].value_counts()
valid_cities = city_counts[city_counts >= 10].index
df_sell['city_encoded'] = df_sell['city_slug'].where(df_sell['city_slug'].isin(valid_cities), 'سایر')
df_sell['city_encoded'] = le_city.fit_transform(df_sell['city_encoded'])

# ---------- 9. ویژگی‌های نهایی (5 مورد — کمینه و هوشمندانه) ----------
features = [
    'price',          # قیمت فروش (تومان)
    'size',           # زیربنا (متر)
    'rooms_num',      # تعداد اتاق
    'year',           # سال ساخت — ✅ جایگزین amenity
    'city_encoded'    # شهر — ✅ جایگزین UTM (تفسیرپذیرتر)
]

X = df_sell[features].copy().dropna()

print(f"✅ پس از حذف NaN: {len(X)} رکورد")

# ---------- 10. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 11. ✅ KMeans با K=10 ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_scaled)
df_sell = df_sell.loc[X.index].copy()  # هماهنگی ایندکس
df_sell['cluster'] = clusters

# ---------- 12. رسم: قیمت vs زیربنا ----------
df_sell['price_mil'] = df_sell['price'] / 1_000_000

fig = px.scatter(
    df_sell,
    x=lat_utm,
    y=lon_utm,
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={
        'size': 'زیربنا (متر مربع)',
        'price_mil': 'قیمت فروش (میلیون تومان)',
        'cluster': 'خوشه'
    },
    title='خوشه‌بندی KMeans — فقط فروشی‌ها (5 ویژگی معنادار)',
    opacity=0.65,
    width=950, height=650
)

# مراکز خوشه‌ها
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)
centers_df = pd.DataFrame(centers_original, columns=features)

fig.add_trace(go.Scatter(
    x=centers_df['size'],
    y=centers_df['price'] / 1_000_000,
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(10)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=18)
fig.write_html("kmeans_sell_optimized.html")
fig.show()

# ---------- 13. خلاصه خوشه‌ها ----------
summary = df_sell.groupby('cluster').agg(
    count=('price', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_year=('year', 'mean'),
    main_city=('city_slug', lambda x: x.mode().iloc[0] if not x.mode().empty else '—')
).round(2)

print("\n📊 خلاصه ۱۰ خوشه (با سال ساخت و شهر):")
print(summary.sort_values('avg_price_mil', ascending=False))

# ---------- 14. تحلیل خوشه‌های کلیدی ----------
high_year = summary[summary['avg_year'] > 1395]
old_year = summary[summary['avg_year'] < 1370]

print("\n🔍 یافته‌های جالب:")
if not high_year.empty:
    print(f"• خانه‌های جدیدساخت ({high_year.index.tolist()}): میانگین سال = {high_year['avg_year'].mean():.0f}")
if not old_year.empty:
    print(f"• خانه‌های قدیمی ({old_year.index.tolist()}): میانگین سال = {old_year['avg_year'].mean():.0f}")

In [ ]:
# --------------------------------------------------
# ✅ KMeans + PCA — فقط آگهی‌های فروشی + سال ساخت + شهر
#    ✅ بدون transformable_price
#    ✅ با PCA برای استخراج سیگنال اصلی
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go


df = data.copy()

# ---------- 2. تبدیل اعداد فارسی ----------
def persian_to_english_digits(text):
    if not isinstance(text, str): return text
    return text.translate(str.maketrans("۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩", "01234567890123456789"))





df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. پاک‌سازی سال ساخت ----------
def parse_year(val):
    if pd.isna(val): return np.nan
    s = str(val).strip()
    s = persian_to_english_digits(s)
    nums = re.findall(r'1[3-4]\d{2}', s)
    return int(nums[0]) if nums else np.nan

df['year'] = df['construction_year'].apply(parse_year)

# ---------- 5. فیلتر فقط آگهی‌های فروشی ----------
df_sell = df[
    (df['cat2_slug'].str.contains('sell', case=False, na=False)) &
    (df['price_value'].notna()) &
    (df['building_size'].notna()) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
].copy()

# ---------- 6. تبدیل عددی + فیلتر منطقی ----------
df_sell['price'] = pd.to_numeric(df_sell['price_value'], errors='coerce')
df_sell['size'] = pd.to_numeric(df_sell['building_size'], errors='coerce')

df_sell = df_sell[
    (df_sell['price'] > 10_000_000) & (df_sell['price'] <= 200_000_000_000) &
    (df_sell['size'] > 10) & (df_sell['size'] <= 50_000) &
    (df_sell['year'].between(1300, 1405))
]

print(f"✅ آگهی‌های فروشی معتبر: {len(df_sell)}")

# ---------- 7. کدگذاری شهر ----------
le_city = LabelEncoder()
city_counts = df_sell['city_slug'].value_counts()
valid_cities = city_counts[city_counts >= 10].index
df_sell['city_encoded'] = df_sell['city_slug'].where(df_sell['city_slug'].isin(valid_cities), 'سایر')
df_sell['city_encoded'] = le_city.fit_transform(df_sell['city_encoded'])

# ---------- 8. ویژگی‌های نهایی (5 مورد) ----------
features = [
    'price',          # قیمت فروش
    'size',           # زیربنا
    'rooms_num',      # تعداد اتاق
    'year',           # سال ساخت ✅
    'city_encoded'    # شهر ✅
]

X = df_sell[features].copy().dropna()
print(f"✅ پس از حذف NaN: {len(X)} رکورد")

# ---------- 9. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 10. PCA (90% واریانس) ----------
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_comp = np.argmax(cumsum >= 0.90) + 1

pca = PCA(n_components=n_comp, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"\n📊 PCA: {n_comp} مؤلفه برای 90% واریانس")
print("وزن‌های مؤلفه اول (PC1):")
pc1_weights = pd.Series(pca.components_[0], index=features).round(3)
print(pc1_weights)

# ---------- 11. ✅ KMeans با K=10 روی فضای PCA ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_pca)
df_sell = df_sell.loc[X.index].copy()
df_sell['cluster'] = clusters

# ---------- 12. رسم در فضای PCA ----------
df_plot = df_sell.copy()
df_plot['PC1'] = X_pca[:, 0]
df_plot['PC2'] = X_pca[:, 1] if X_pca.shape[1] > 1 else 0
df_plot['price_mil'] = df_plot['price'] / 1_000_000

fig = px.scatter(
    df_plot,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={'PC1': 'مؤلفه اصلی ۱', 'PC2': 'مؤلفه اصلی ۲'},
    title=f'خوشه‌بندی KMeans (K=10) روی فضای PCA — {n_comp} مؤلفه (90% واریانس)',
    opacity=0.65,
    width=950, height=650
)

# مراکز خوشه‌ها
centers_pca = kmeans.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers_pca[:, 0], 
    y=centers_pca[:, 1] if centers_pca.shape[1] > 1 else [0]*len(centers_pca),
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(10)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=18)
fig.write_html("kmeans_pca_optimized.html")
fig.show()

# ---------- 13. خلاصه خوشه‌ها ----------
summary = df_plot.groupby('cluster').agg(
    count=('price', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_year=('year', 'mean'),
    main_city=('city_slug', lambda x: x.mode().iloc[0] if not x.mode().empty else '—')
).round(2)

print("\n📊 خلاصه ۱۰ خوشه (با PCA + سال ساخت + شهر):")
print(summary.sort_values('avg_price_mil', ascending=False))

# ---------- 14. تفسیر مؤلفه‌ها ----------
print("\n🔍 تفسیر مؤلفه‌های PCA:")
if n_comp >= 2:
    pc2_weights = pd.Series(pca.components_[1], index=features).round(3)
    print("PC1 (اولین مؤلفه):")
    print(pc1_weights.to_string())
    print("\nPC2 (دومین مؤلفه):")
    print(pc2_weights.to_string())
    
    # تحلیل خوشه‌های قدیمی vs جدید
    old_clusters = summary[summary['avg_year'] < 1370].index
    new_clusters = summary[summary['avg_year'] > 1395].index
    if len(old_clusters) > 0 and len(new_clusters) > 0:
        print(f"\n✅ یافته: خوشه‌های قدیمی ({list(old_clusters)}) و جدید ({list(new_clusters)}) در فضای PCA به‌وضوح تفکیک شده‌اند.")

In [ ]:
# --------------------------------------------------
# ✅ KMeans + PCA — با تمام فیچرهای پیشنهادی شما
#    ✅ فقط فروشی‌ها
#    ✅ 11 ویژگی معنادار (مالی + مکانی + امکانات)
#    ✅ K=10 دقیقاً مطابق صورت سوال
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go


df = data.copy()

# ---------- 2. تبدیل اعداد فارسی ----------
def persian_to_english_digits(text):
    if not isinstance(text, str): return text
    return text.translate(str.maketrans("۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩", "01234567890123456789"))




df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. پاک‌سازی سال ساخت ----------
def parse_year(val):
    if pd.isna(val): return np.nan
    s = str(val).strip()
    s = persian_to_english_digits(s)
    nums = re.findall(r'1[3-4]\d{2}', s)
    return int(nums[0]) if nums else np.nan

df['year'] = df['construction_year'].apply(parse_year)

# ---------- 5. فیلتر فقط آگهی‌های فروشی ----------
df_sell = df[
    (df['cat2_slug'].str.contains('sell', case=False, na=False)) &
    (df['price_value'].notna()) &
    (df['building_size'].notna()) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
].copy()

# ---------- 6. تبدیل عددی + فیلتر منطقی ----------
df_sell['price'] = pd.to_numeric(df_sell['price_value'], errors='coerce')
df_sell['size'] = pd.to_numeric(df_sell['building_size'], errors='coerce')

df_sell = df_sell[
    (df_sell['price'] > 10_000_000) & (df_sell['price'] <= 200_000_000_000) &
    (df_sell['size'] > 10) & (df_sell['size'] <= 50_000) &
    (df_sell['year'].between(1300, 1405))
]

print(f"✅ آگهی‌های فروشی معتبر: {len(df_sell)}")

# ---------- 7. تبدیل امکانات به باینری (0/1) ----------
bool_cols = [
    'has_pool', 'has_jacuzzi', 'has_elevator',
    'has_warehouse', 'has_parking', 'has_security_guard'
]

for col in bool_cols:
    df_sell[col] = df_sell[col].astype(str).str.lower().isin(['true', '1', 'yes']).astype(int)

# ---------- 8. کدگذاری شهر ----------
le_city = LabelEncoder()
city_counts = df_sell['city_slug'].value_counts()
valid_cities = city_counts[city_counts >= 10].index
df_sell['city_encoded'] = df_sell['city_slug'].where(df_sell['city_slug'].isin(valid_cities), 'سایر')
df_sell['city_encoded'] = le_city.fit_transform(df_sell['city_encoded'])

# ---------- 9. ویژگی‌های نهایی (11 مورد) ----------
features = [
    'price', 'size', 'rooms_num', 'year', 'city_encoded',
    'has_pool', 'has_jacuzzi', 'has_elevator',
    'has_warehouse', 'has_parking', 'has_security_guard'
]

X = df_sell[features].copy().dropna()
print(f"✅ پس از حذف NaN: {len(X)} رکورد")

# ---------- 10. StandardScaler ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 11. PCA (90% واریانس) ----------
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_comp = np.argmax(cumsum >= 0.90) + 1

pca = PCA(n_components=n_comp, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"\n📊 PCA: {n_comp} مؤلفه برای 90% واریانس")
print("توضیح واریانس هر مؤلفه:", pca.explained_variance_ratio_.round(3))

# ---------- 12. ✅ KMeans با K=10 ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_pca)
df_sell = df_sell.loc[X.index].copy()
df_sell['cluster'] = clusters

# ---------- 13. رسم در فضای PCA ----------
df_plot = df_sell.copy()
df_plot['PC1'] = X_pca[:, 0]
df_plot['PC2'] = X_pca[:, 1] if X_pca.shape[1] > 1 else 0
df_plot['price_mil'] = df_plot['price'] / 1_000_000

fig = px.scatter(
    df_plot,
    x='PC1',
    y='PC2',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={'PC1': 'مؤلفه اصلی ۱', 'PC2': 'مؤلفه اصلی ۲'},
    title=f'خوشه‌بندی KMeans (K=10) — 11 ویژگی + PCA ({n_comp} مؤلفه)',
    opacity=0.6,
    width=950, height=650
)

# مراکز خوشه‌ها
centers_pca = kmeans.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers_pca[:, 0], 
    y=centers_pca[:, 1] if centers_pca.shape[1] > 1 else [0]*len(centers_pca),
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(10)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=18)
fig.write_html("kmeans_pca_full_features.html")
fig.show()

# ---------- 14. خلاصه خوشه‌ها ----------
summary = df_plot.groupby('cluster').agg(
    count=('price', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_year=('year', 'mean'),
    main_city=('city_slug', lambda x: x.mode().iloc[0] if not x.mode().empty else '—'),
    pool=('has_pool', 'mean'),
    jacuzzi=('has_jacuzzi', 'mean'),
    elevator=('has_elevator', 'mean'),
    parking=('has_parking', 'mean'),
    guard=('has_security_guard', 'mean')
).round(3)

print("\n📊 خلاصه ۱۰ خوشه (با تمام امکانات):")
print(summary.sort_values('avg_price_mil', ascending=False))

# ---------- 15. تحلیل خوشه‌های لوکس ----------
luxury_mask = (summary['pool'] > 0.3) | (summary['jacuzzi'] > 0.2)
luxury_clusters = summary[luxury_mask].index.tolist()
if luxury_clusters:
    print(f"\n🌟 خوشه‌های لوکس (دارای استخر/جکوزی): {luxury_clusters}")
    for c in luxury_clusters:
        row = summary.loc[c]
        print(f"  • C{c}: قیمت={row['avg_price_mil']:.0f}M, استخر={row['pool']:.0%}, جکوزی={row['jacuzzi']:.0%}")

In [ ]:
# --------------------------------------------------
# ✅ KMeans بدون PCA — با تمام 11 ویژگی پیشنهادی شما
#    ✅ فقط آگهی‌های فروشی
#    ✅ 6 امکانات باینری (استخر، جکوزی، آسانسور، ...)
#    ✅ K=10 دقیقاً مطابق صورت سوال
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go


df = data.copy()





df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# ---------- 4. پاک‌سازی سال ساخت ----------
def parse_year(val):
    if pd.isna(val): return np.nan
    s = str(val).strip()
    s = persian_to_english_digits(s)
    nums = re.findall(r'1[3-4]\d{2}', s)
    return int(nums[0]) if nums else np.nan

df['year'] = df['construction_year'].apply(parse_year)

# ---------- 5. فیلتر فقط آگهی‌های فروشی ----------
df_sell = df[
    (df['cat2_slug'].str.contains('sell', case=False, na=False)) &
    (df['price_value'].notna()) &
    (df['building_size'].notna()) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
].copy()

# ---------- 6. تبدیل عددی + فیلتر منطقی ----------
df_sell['price'] = pd.to_numeric(df_sell['price_value'], errors='coerce')
df_sell['size'] = pd.to_numeric(df_sell['building_size'], errors='coerce')

df_sell = df_sell[
    (df_sell['price'] > 10_000_000) & (df_sell['price'] <= 200_000_000_000) &
    (df_sell['size'] > 10) & (df_sell['size'] <= 50_000) &
    (df_sell['year'].between(1300, 1405))
]

print(f"✅ آگهی‌های فروشی معتبر: {len(df_sell)}")

# ---------- 7. تبدیل امکانات به باینری (0/1) ----------
bool_cols = [
    'has_pool', 'has_jacuzzi', 'has_elevator',
    'has_warehouse', 'has_parking', 'has_security_guard'
]

for col in bool_cols:
    df_sell[col] = df_sell[col].astype(str).str.lower().isin(['true', '1', 'yes']).astype(int)

# ---------- 8. کدگذاری شهر ----------
le_city = LabelEncoder()
city_counts = df_sell['city_slug'].value_counts()
valid_cities = city_counts[city_counts >= 10].index
df_sell['city_encoded'] = df_sell['city_slug'].where(df_sell['city_slug'].isin(valid_cities), 'سایر')
df_sell['city_encoded'] = le_city.fit_transform(df_sell['city_encoded'])

# ---------- 9. ویژگی‌های نهایی (11 مورد) ----------
features = [
    'price', 'size', 'rooms_num', 'year', 'city_encoded',
    'has_pool', 'has_jacuzzi', 'has_elevator',
    'has_warehouse', 'has_parking', 'has_security_guard'
]

X = df_sell[features].copy().dropna()
print(f"✅ پس از حذف NaN: {len(X)} رکورد")

# ---------- 10. ✅ StandardScaler (حیاتی!) ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 11. ✅ KMeans با K=10 (بدون PCA) ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_scaled)
df_sell = df_sell.loc[X.index].copy()
df_sell['cluster'] = clusters

# ---------- 12. رسم: قیمت vs زیربنا (با 10 خوشه) ----------
df_sell['price_mil'] = df_sell['price'] / 1_000_000

fig = px.scatter(
    df_sell,
    x='size',
    y='price_mil',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={
        'size': 'زیربنا (متر مربع)',
        'price_mil': 'قیمت فروش (میلیون تومان)',
        'cluster': 'خوشه'
    },
    title='خوشه‌بندی KMeans (K=10) — بدون PCA، با 11 ویژگی معنادار',
    opacity=0.65,
    width=950, height=650
)

# مراکز خوشه‌ها (در فضای اصلی)
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)
centers_df = pd.DataFrame(centers_original, columns=features)

fig.add_trace(go.Scatter(
    x=centers_df['size'],
    y=centers_df['price'] / 1_000_000,
    mode='markers+text',
    marker=dict(size=18, color='black', symbol='star', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(10)],
    textposition="top center",
    name='مراکز خوشه‌ها'
))

fig.update_layout(font=dict(size=13), title_font_size=18)
fig.write_html("kmeans_no_pca_full_features.html")
fig.show()

# ---------- 13. خلاصه خوشه‌ها ----------
summary = df_sell.groupby('cluster').agg(
    count=('price', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('size', 'mean'),
    avg_rooms=('rooms_num', 'mean'),
    avg_year=('year', 'mean'),
    main_city=('city_slug', lambda x: x.mode().iloc[0] if not x.mode().empty else '—'),
    pool=('has_pool', 'mean'),
    jacuzzi=('has_jacuzzi', 'mean'),
    elevator=('has_elevator', 'mean'),
    warehouse=('has_warehouse', 'mean'),
    parking=('has_parking', 'mean'),
    guard=('has_security_guard', 'mean')
).round(3)

print("\n📊 خلاصه ۱۰ خوشه (بدون PCA — با تمام امکانات):")
print(summary.sort_values('avg_price_mil', ascending=False))

# ---------- 14. تحلیل خوشه‌های لوکس ----------
luxury_mask = (summary['pool'] > 0.3) | (summary['jacuzzi'] > 0.2)
luxury_clusters = summary[luxury_mask].index.tolist()
if luxury_clusters:
    print(f"\n🌟 خوشه‌های لوکس (دارای استخر و/یا جکوزی): {luxury_clusters}")
    for c in luxury_clusters:
        row = summary.loc[c]
        print(f"  • C{c}: قیمت={row['avg_price_mil']:.0f}M | استخر={row['pool']:.0%} | جکوزی={row['jacuzzi']:.0%}")

In [ ]:
# --------------------------------------------------
# 🧠 Feature Engineering + Feature Selection → KMeans (K=10)
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")


df = data.copy()


# فقط فروشی‌ها
df = df[df['cat2_slug'].str.contains('sell', case=False, na=False)].copy()

# تبدیل عددی
for col in ['price_value', 'building_size', 'location_latitude', 'location_longitude']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# فیلتر معتبر
df = df[
    (df['price_value'] > 10_000_000) & (df['price_value'] <= 200_000_000_000) &
    (df['building_size'] > 10) & (df['building_size'] <= 50_000) &
    (df['location_latitude'].between(25, 40)) &
    (df['location_longitude'].between(44, 63))
]

print(f"✅ آگهی‌های فروشی معتبر: {len(df)}")

# ---------- 2. Feature Engineering ----------
# a) rooms_num
def parse_rooms(val):
    if pd.isna(val): return np.nan
    s = str(val).lower()
    if 'بدون' in s: return 0
    if 'یک' in s: return 1
    if 'دو' in s: return 2
    if 'سه' in s: return 3
    if 'چهار' in s: return 4
    if 'پنج' in s: return 5
    nums = re.findall(r'\d+', s)
    return int(nums[0]) if nums else np.nan

df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

# b) year
def parse_year(val):
    if pd.isna(val): return np.nan
    s = str(val)
    s = persian_to_english_digits(s)
    nums = re.findall(r'1[3-4]\d{2}', s)
    return int(nums[0]) if nums else np.nan

df['year'] = df['construction_year'].apply(parse_year)

# c) امکانات باینری
bool_cols = ['has_pool', 'has_jacuzzi', 'has_elevator', 
             'has_warehouse', 'has_parking', 'has_security_guard']
for col in bool_cols:
    df[col] = df[col].astype(str).str.lower().isin(['true', '1', 'yes']).astype(int)

# d) ✅ ویژگی‌های مهندسی‌شده (کلید موفقیت!)
df['price_per_sqm'] = df['price_value'] / df['building_size']          # قیمت هر متر
df['age'] = 1405 - df['year']                                         # سن ملک
df['luxury_score'] = df[['has_pool', 'has_jacuzzi']].sum(axis=1)      # امتیاز لوکس (0-2)
df['basic_score'] = df[['has_elevator', 'has_parking', 'has_security_guard']].sum(axis=1)  # امتیاز پایه (0-3)

# e) شهر (فقط شهرهای مهم)
le_city = LabelEncoder()
city_counts = df['city_slug'].value_counts()
valid_cities = city_counts[city_counts >= 10].index
df['city_encoded'] = df['city_slug'].where(df['city_slug'].isin(valid_cities), 'سایر')
df['city_encoded'] = le_city.fit_transform(df['city_encoded'])

# ---------- 3. انتخاب ویژگی‌های اولیه ----------
base_features = [
    'price_value', 'building_size', 'rooms_num', 'age',
    'luxury_score', 'basic_score', 'city_encoded'
]

X = df[base_features].dropna()
y = X['price_value']  # برای mutual info
X = X.drop(columns=['price_value'])

print(f"✅ پس از feature engineering: {len(X)} رکورد × {X.shape[1]} ویژگی")

# ---------- 4. Feature Selection — سه مرحله هوشمندانه ----------

# مرحله 1: حذف ویژگی‌های با واریانس بسیار کم
selector_var = VarianceThreshold(threshold=0.01)
X_var = selector_var.fit_transform(X)
selected_var = X.columns[selector_var.get_support()]
print(f"پس از VarianceThreshold: {X_var.shape[1]} ویژگی")

# مرحله 2: امتیازدهی با Mutual Information
mi_scores = mutual_info_regression(X_var, y[X.index], random_state=42)
mi_df = pd.DataFrame({'feature': selected_var, 'mi': mi_scores}).sort_values('mi', ascending=False)
print("\nامتیاز MI (بالاتر = مهم‌تر):")
print(mi_df)

# مرحله 3: انتخاب بهینه با RFE + Random Forest
rf = RandomForestRegressor(n_estimators=50, random_state=42)
rfe = RFE(rf, n_features_to_select=1)  # برای رتبه‌بندی
rfe.fit(X_var, y[X.index])
rfe_ranking = pd.DataFrame({
    'feature': selected_var,
    'rank': rfe.ranking_
}).sort_values('rank')

print("\nرتبه‌بندی RFE (پایین‌تر = مهم‌تر):")
print(rfe_ranking)

# ---------- 5. تعیین تعداد بهینه ویژگی ----------
best_k = None
best_score = -1
scores = []

for k in range(2, len(selected_var)+1):
    # انتخاب k ویژگی برتر (ترکیب MI + RFE)
    top_k = mi_df.head(k)['feature'].tolist()
    X_k = X[top_k]
    
    # scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_k)
    
    # KMeans با K=10
    kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    
    # Silhouette Score
    sil = silhouette_score(X_scaled, labels, sample_size=min(5000, len(X_scaled)))
    scores.append((k, sil))
    
    if sil > best_score:
        best_score = sil
        best_k = k

print(f"\n✅ بهترین تعداد ویژگی: {best_k} → Silhouette = {best_score:.4f}")

# رسم
k_vals, sil_vals = zip(*scores)
fig = px.line(x=k_vals, y=sil_vals, markers=True,
              labels={'x': 'تعداد ویژگی‌ها', 'y': 'Silhouette Score'},
              title='تعیین تعداد بهینه ویژگی‌ها')
fig.add_vline(x=best_k, line_dash="dash", line_color="red", 
              annotation_text=f"بهینه: K={best_k}", annotation_position="top right")
fig.show()

# ---------- 6. انتخاب نهایی ویژگی‌ها ----------
final_features = mi_df.head(best_k)['feature'].tolist()
print(f"\n✅ ویژگی‌های نهایی ({len(final_features)} مورد):")
print(final_features)

X_final = X[final_features].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

# ---------- 7. ✅ KMeans با K=10 ----------
kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_scaled)
df_final = df.loc[X_final.index].copy()
df_final['cluster'] = clusters

# ---------- 8. رسم نتیجه ----------
df_final['price_mil'] = df_final['price_value'] / 1_000_000

fig = px.scatter(
    df_final,
    x='building_size',
    y='price_mil',
    color='cluster',
    color_continuous_scale='Rainbow',
    labels={
        'building_size': 'زیربنا (متر مربع)',
        'price_mil': 'قیمت (میلیون تومان)',
        'cluster': 'خوشه'
    },
    title=f'خوشه‌بندی نهایی (K=10) — با {best_k} ویژگی بهینه‌شده',
    opacity=0.65,
    width=950, height=650
)

# مراکز
centers = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=final_features)
if 'building_size' in centers_df.columns and 'price_value' in df_final.columns:
    fig.add_trace(go.Scatter(
        x=centers_df['building_size'] if 'building_size' in centers_df.columns else [0]*10,
        y=[c / 1e6 for c in kmeans.cluster_centers_[:, final_features.index('building_size')] 
           if 'building_size' in final_features else [0]*10],
        mode='markers+text',
        marker=dict(size=18, color='black', symbol='star'),
        text=[f'C{i}' for i in range(10)],
        textposition="top center",
        showlegend=False
    ))

fig.update_layout(font=dict(size=13))
fig.write_html("kmeans_optimized_features.html")
fig.show()

# ---------- 9. خلاصه خوشه‌ها ----------
summary = df_final.groupby('cluster').agg(
    count=('price_value', 'size'),
    avg_price_mil=('price_mil', 'mean'),
    avg_size=('building_size', 'mean'),
    avg_age=('age', 'mean'),
    avg_luxury=('luxury_score', 'mean'),
    avg_basic=('basic_score', 'mean'),
    main_city=('city_slug', lambda x: x.mode().iloc[0] if not x.mode().empty else '—')
).round(2)

print("\n📊 خلاصه ۱۰ خوشه (با ویژگی‌های بهینه‌شده):")
print(summary.sort_values('avg_price_mil', ascending=False))

In [ ]:
# --------------------------------------------------
# 🧠 Auto Feature Selection + KMeans (K=10)
#    ✅ همه ستون‌ها ورودی هستند
#    ✅ بدون فرض اولیه
#    ✅ Imputation صحیح (بدون Shape mismatch)
# --------------------------------------------------
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")


df = data.copy()
print(f"✅ داده اولیه: {df.shape[0]:,} ردیف × {df.shape[1]} ستون")



# ---------- 2. پاک‌سازی اولیه ----------
df = df.dropna(thresh=int(0.05 * len(df)), axis=1)  # حذف ستون‌های >95% NaN
nunique = df.nunique()
df = df.drop(columns=nunique[nunique == 1].index)  # حذف ثابت‌ها
print(f"پس از پاک‌سازی اولیه: {df.shape[1]} ستون")

# ---------- 3. تبدیل به انواع مناسب ----------
# a) عددی
numeric_cols = []
for col in df.columns:
    try:
        pd.to_numeric(df[col], errors='raise')
        numeric_cols.append(col)
    except:
        pass

df_num = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# b) باینری
binary_cols = []
for col in df.columns:
    if col not in numeric_cols:
        vals = set(str(x).strip().lower() for x in df[col].dropna().unique())
        if vals <= {'true', 'false', '0', '1', 'yes', 'no', 'unselect'}:
            binary_cols.append(col)

df_bin = pd.DataFrame()
for col in binary_cols:
    df_bin[col + '_bin'] = df[col].astype(str).str.lower().isin(['true', '1', 'yes']).astype(int)

# c) دسته‌ای (فقط ستون‌های موجود و با کاردینالیته معقول)
cat_cols_candidate = ['city_slug', 'cat2_slug', 'cat3_slug', 'neighborhood_slug', 'deed_type']
cat_cols = [col for col in cat_cols_candidate if col in df.columns and 2 <= df[col].nunique() <= 50]

df_cat = pd.DataFrame(index=df.index)
for col in cat_cols:
    le = LabelEncoder()
    series = df[col].fillna('ناشناس').astype(str)
    df_cat[col + '_enc'] = le.fit_transform(series)

# d) Feature Engineering (خودکار)
engineered = pd.DataFrame(index=df.index)
if 'price_value' in df_num.columns and 'building_size' in df_num.columns:
    p = df_num['price_value']
    s = df_num['building_size']
    engineered['price_per_sqm'] = p / s
    engineered['log_price'] = np.log1p(p)
    engineered['log_size'] = np.log1p(s)

# ---------- 4. ترکیب ویژگی‌ها ----------
X_raw = pd.concat([df_num, df_bin, df_cat, engineered], axis=1)
print(f"✅ پس از مهندسی: {X_raw.shape[1]} ویژگی")

# حذف همبستگی بالا
corr = X_raw.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.95)]
X_raw = X_raw.drop(columns=to_drop)
print(f"پس از حذف همبستگی بالا: {X_raw.shape[1]} ویژگی")

# ---------- 5. Imputation صحیح (بدون Shape mismatch) ----------
# جدا کردن y (اولویت: price_value)
# ---------- Imputation — روش قطعی (بدون Shape mismatch) ----------
# جدا کردن y و هماهنگی ایندکس
y = None
X_temp = X_raw.copy()

for col in ['price_value', 'transformable_price']:
    if col in X_temp.columns:
        # تبدیل به عددی و پیدا کردن مقادیر معتبر
        y_series = pd.to_numeric(X_temp[col], errors='coerce')
        mask = y_series.notna()
        y = y_series[mask]
        X_temp = X_temp.loc[mask].drop(columns=[col])
        print(f"🎯 هدف: '{col}' → {len(y):,} نمونه")
        break

if y is None:
    # fallback: اولین ستون عددی
    for col in X_temp.columns:
        y_series = pd.to_numeric(X_temp[col], errors='coerce')
        mask = y_series.notna()
        if mask.sum() > 100:  # حداقل 100 نمونه
            y = y_series[mask]
            X_temp = X_temp.loc[mask].drop(columns=[col])
            print(f"⚠️ هدف پیش‌فرض: '{col}'")
            break

# ✅ روش قطعی: Imputation + فیلتر ستون‌های موجود
imputer = SimpleImputer(strategy='median')
X_imputed_arr = imputer.fit_transform(X_temp)

# 🔑 استفاده از ستون‌هایی که واقعاً در خروجی هستند
mask_cols = ~np.isnan(imputer.statistics_)  # ستون‌هایی که imputer روی آن‌ها کار کرده
selected_cols = X_temp.columns[mask_cols]

X_imputed = pd.DataFrame(
    X_imputed_arr,
    columns=selected_cols,  # ✅ فقط ستون‌های باقی‌مانده
    index=X_temp.index
)

print(f"پس از Imputation: {X_imputed.shape[1]} ویژگی (از {X_temp.shape[1]})")
# ---------- 6. Feature Selection خودکار ----------
# Stage 1: VarianceThreshold
selector_var = VarianceThreshold(threshold=0.01)
X_var = pd.DataFrame(
    selector_var.fit_transform(X_imputed),
    columns=X_imputed.columns[selector_var.get_support()],
    index=X_imputed.index
)

# Stage 2: SelectKBest
if X_var.shape[1] > 0:
    k_best = min(20, X_var.shape[1])
    selector_k = SelectKBest(score_func=f_regression, k=k_best)
    X_kbest = pd.DataFrame(
        selector_k.fit_transform(X_var, y),
        columns=X_var.columns[selector_k.get_support()],
        index=X_var.index
    )
    print(f"پس از SelectKBest: {X_kbest.shape[1]} ویژگی")
else:
    X_kbest = X_var.copy()

# Stage 3: RFE برای رتبه‌بندی
if not X_kbest.empty:
    rf = RandomForestRegressor(n_estimators=50, random_state=42)
    rfe = RFE(rf, n_features_to_select=min(10, len(X_kbest.columns)), step=1)
    rfe.fit(X_kbest, y)
    ranking = pd.DataFrame({
        'feature': X_kbest.columns,
        'rank': rfe.ranking_
    }).sort_values('rank')
else:
    ranking = pd.DataFrame(columns=['feature', 'rank'])

# ---------- 7. تعیین تعداد بهینه ویژگی ----------
best_k, best_score = 1, -1
scores = []

for k in range(1, min(11, len(ranking)+1)):
    top_k = ranking.head(k)['feature'].tolist()
    if not top_k: break
    X_k = X_kbest[top_k]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_k)
    
    kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels, sample_size=min(5000, len(X_scaled)), random_state=42)
    scores.append((k, sil))
    if sil > best_score:
        best_score, best_k = sil, k

print(f"\n✅ بهترین تعداد ویژگی: {best_k} → Silhouette = {best_score:.4f}")

# رسم
if scores:
    k_vals, sil_vals = zip(*scores)
    fig = px.line(x=k_vals, y=sil_vals, markers=True,
                  labels={'x': 'تعداد ویژگی', 'y': 'Silhouette'},
                  title='انتخاب تعداد بهینه ویژگی')
    fig.add_vline(x=best_k, line_dash="dash", line_color="red")
    fig.show()

# ---------- 8. KMeans نهایی (K=10) ----------
final_features = ranking.head(best_k)['feature'].tolist() if not ranking.empty else X_kbest.columns[:best_k].tolist()
print(f"\n✅ ویژگی‌های نهایی ({len(final_features)}):")
for i, f in enumerate(final_features, 1):
    print(f"{i}. {f}")

X_final = X_kbest[final_features].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

kmeans = KMeans(n_clusters=10, random_state=42, n_init=25)
clusters = kmeans.fit_predict(X_scaled)

# ---------- 9. خروجی ----------
df_out = df.loc[X_final.index].copy()
df_out['cluster'] = clusters

summary = df_out.groupby('cluster').agg(
    count=('cluster', 'size'),
    **{f: (f, 'mean') for f in final_features if f in df_out.columns}
).round(2)

print("\n📊 خلاصه ۱۰ خوشه:")
print(summary)

# Part 1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer


In [ ]:
df=data.copy()

In [ ]:
df['rooms_num'] = df['rooms_count'].apply(parse_rooms).fillna(2)

In [ ]:
features = [
    "price_value",
    "building_size",
    "rooms_num",
    "construction_year",
    "location_latitude",
    "location_longitude",
    "city_slug"
]

df_model = df[features].copy()


In [ ]:
df_model.info()

In [ ]:
df_model = df_model.dropna(
    subset=["location_latitude","location_longitude","city_slug"]
).reset_index(drop=True)

In [ ]:
df_model.info()

In [ ]:
num_cols = [
    "price_value",
    "building_size",
    "rooms_num",
    "construction_year"
]

geo_features=[
    "location_latitude",
    "location_longitude"
]


In [ ]:
for col in num_cols:
    df_model[col] = pd.to_numeric(df_model[col], errors="coerce")

df_model = df_model.dropna()


In [ ]:
for col in geo_features:
    df_model[col] = pd.to_numeric(df_model[col], errors="coerce")

df_model = df_model.dropna()

In [ ]:
df_model_sell = df_model[
    (df_model['price_value']>0) &
    (df_model['building_size'].notna()) &
    (df_model['location_latitude'].between(25, 40)) &
    (df_model['location_longitude'].between(44, 63))
].copy()

In [ ]:
df_model_sell.info()

In [ ]:
city_mean_price = (
    df_model_sell.groupby("city_slug")["price_value"]
            .mean()
)

df_model_sell["city_encoded"] = df_model_sell["city_slug"].map(city_mean_price)


In [ ]:
df_model_sell.info()

In [ ]:


imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()

X_num = imputer.fit_transform(df_model_sell[num_cols])
X_num = scaler.fit_transform(X_num)


# X_geo = imputer.fit_transform(df_model_sell[geo_features])
X_geo = scaler.fit_transform(df_model_sell[geo_features])


city_encoded = imputer.fit_transform(
    df_model_sell[["city_encoded"]]
)
city_encoded = scaler.fit_transform(city_encoded)


In [ ]:
# X_geo = scaler.fit_transform(geo_features)

In [ ]:
weights = np.array([3, 1.5, 1, 1])  
X_weighted = X_num * weights

In [ ]:
city_weight = 0.3

X_final = np.hstack([
    X_num,
    X_geo,
    city_encoded * city_weight
])


In [ ]:
kmeans = KMeans(
    n_clusters=10,
    random_state=42,
    n_init=10
)

df_model_sell["cluster"] = kmeans.fit_predict(X_final)


In [ ]:
R = 6378137  # شعاع زمین (متر)

lat_rad = np.deg2rad(df_model_sell["location_latitude"])
lon_rad = np.deg2rad(df_model_sell["location_longitude"])

lat_mean = lat_rad.mean()

df_model_sell["utm_x"] = R * lon_rad * np.cos(lat_mean)
df_model_sell["utm_y"] = R * lat_rad


In [ ]:
# centers_scaled = kmeans.cluster_centers_
# centers = scaler.inverse_transform(centers_scaled)

# center_lat = np.deg2rad(centers[:, features.index("location_latitude")])
# center_lon = np.deg2rad(centers[:, features.index("location_longitude")])

# center_x = R * center_lon * np.cos(lat_mean)
# center_price = centers[:, features.index("price_value")]


In [ ]:
# plt.figure(figsize=(11, 7))

# scatter = plt.scatter(
#     data["utm_x"],
#     data["price_value"],
#     c=data["cluster"],
#     alpha=0.7,
#     marker="o"
# )

# # مراکز خوشه‌ها
# plt.scatter(
#     center_x,
#     center_price,
#     marker="X",
#     s=250
# )

# plt.xlabel("UTM X Coordinate (meters)")
# plt.ylabel("Transformed Price")
# plt.title("K-Means Clustering of Properties (k=10)")

# plt.colorbar(scatter, label="Cluster ID")
# plt.grid(True)
# plt.show()


In [ ]:
plt.scatter(
    df_model_sell["utm_x"],
    df_model_sell["utm_y"],
    c=df_model_sell["cluster"],
    alpha=0.7
)

# Middle part

In [ ]:
kmeans = KMeans(
    n_clusters=7,
    random_state=42,
    n_init=10
)

df_model_sell["cluster"] = kmeans.fit_predict(X_final)

In [ ]:
R = 6378137  # شعاع زمین (متر)

lat_rad = np.deg2rad(df_model_sell["location_latitude"])
lon_rad = np.deg2rad(df_model_sell["location_longitude"])

lat_mean = lat_rad.mean()

df_model_sell["utm_x"] = R * lon_rad * np.cos(lat_mean)
df_model_sell["utm_y"] = R * lat_rad

In [ ]:
clusters = np.sort(df_model_sell["cluster"].unique())
for c in clusters:
    cluster_data = df_model_sell[df_model_sell["cluster"] == c]

    plt.scatter(
        df_model_sell["utm_x"],
        df_model_sell["utm_y"],
        c=df_model_sell["cluster"],
        alpha=0.7,
        cmap='tab20',
        label=f"Cluster {c}"
    )

plt.legend(
    title="Clusters",
    markerscale=2,
    fontsize=9,
    title_fontsize=10,
    loc="best"
)

# partIII


In [ ]:
import pyproj
import numpy as np
from sklearn.cluster import DBSCAN
df=data.copy()
df = df.dropna(
    subset=["location_latitude","location_longitude","transformable_price"]
).reset_index(drop=True)
# پروجکشن WGS84 -> UTM Zone 39N (مناسب ایران)
proj_utm = pyproj.Proj(proj="utm", zone=39, ellps="WGS84")

utm_x, utm_y = proj_utm(
    df["location_longitude"].values,
    df["location_latitude"].values
)

df["utm_x"] = utm_x
df["utm_y"] = utm_y


In [ ]:
df_sample = df.sample(
    n=10000,        # یا 30000 بسته به RAM
    random_state=42
)


In [ ]:
X_db = df_sample[["utm_x", "utm_y", "transformable_price"]]

X_db_scaled = StandardScaler().fit_transform(X_db)

dbscan = DBSCAN(
    eps=0.6,
    min_samples=50,
    algorithm="ball_tree"   # مهم
)

df_sample["dbscan_cluster"] = dbscan.fit_predict(X_db_scaled)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))

for c in sorted(df_sample["dbscan_cluster"].unique()):
    subset = df_sample[df_sample["dbscan_cluster"] == c]
    label = "Noise" if c == -1 else f"Cluster {c}"
    
    plt.scatter(
        subset["utm_x"],
        subset["utm_y"],
        s=12,
        alpha=0.6,
        label=label
    )

plt.xlabel("UTM X")
plt.ylabel("UTM Y")
plt.title("DBSCAN Clustering (Sampled Data)")
plt.legend()
plt.show()
